In [1]:
import json
import requests
from bs4 import BeautifulSoup
import spacy
from collections import Counter
from dotenv import load_dotenv
import os
from PyQt5.QtWidgets import QApplication, QWidget, QVBoxLayout, QLabel, QCheckBox, QPushButton, QScrollArea, QVBoxLayout
from openai import OpenAI
import openai

import pandas as pd

In [2]:
import os
from anki.storage import Collection
import random


def get_spanish_dominant_string(str1, str2):
    # Множества букв испанского алфавита и кириллицы
    spanish_alphabet = set("abcdefghijklmnñopqrstuvwxyzáéíóúü")
    cyrillic_alphabet = set("абвгдеёжзийклмнопрстуфхцчшщъыьэюя")

    # Функция для подсчета букв, принадлежащих к определённому алфавиту
    def share_letters_in_alphabet(s, alphabet):
        return sum(1 for char in s.lower() if char in alphabet) / len(s)

    # Подсчёт букв для обеих строк
    spanish_count_str1 = share_letters_in_alphabet(str1, spanish_alphabet)
    cyrillic_count_str1 = share_letters_in_alphabet(str1, cyrillic_alphabet)

    spanish_count_str2 = share_letters_in_alphabet(str2, spanish_alphabet)
    cyrillic_count_str2 = share_letters_in_alphabet(str2, cyrillic_alphabet)

    # Сравнение количества букв
    if spanish_count_str1 > cyrillic_count_str1 and spanish_count_str1 > spanish_count_str2:
        return str1
    elif spanish_count_str2 > cyrillic_count_str2 and spanish_count_str2 > spanish_count_str1:
        return str2
    else:
        return str1

def remove_prefix(s):
    s = s.lower()
    prefixes = ["los ", "los\xa0", "los&nbsp;",
                "las ", "las\xa0", "las&nbsp;",
                "la ", "la\xa0", "la&nbsp;",
                "el ", "el\xa0", "el&nbsp;",]
    for prefix in prefixes:
        if s.startswith(prefix):
            return s[len(prefix):]
    return s


def remove_html_tags_bs(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

# Укажите путь к файлу вашей коллекции Anki (.anki2)
collection_path = os.path.expanduser(r'~/Library/Application Support/Anki2/User 1/collection.anki2')

# Открываем коллекцию
col = Collection(collection_path)

# Получаем ID колоды
# deck_id = col.decks.id(deck_name)

# Получаем все заметки из колоды
note_ids = col.find_notes("deck:Spanish*")
# note_ids = col.find_notes('"deck:Spanish.Others::Inna.20240525.Los colores"')
	

# Список для хранения значений поля "FrontText"
anki_texts = []

# Определяем индексы 'FrontText', 'BackText'
note_example = col.get_note(note_ids[0])
indexes_for_getting_text = []
for index, e in enumerate(note_example.keys()):
    if e in ['FrontText', 'BackText']:
        indexes_for_getting_text.append(index)

# Проходимся по всем заметкам и извлекаем поле "FrontText
for note_id in note_ids:
    note = col.get_note(note_id)

    # Тут надо как-то ещё тип учитывать, но не стал разбираться с документацией
    # if note.note_type()['type'] != 0:
    #     front_text = note.fields[0]  # Предполагается, что первое поле является "FrontText"
    #     front_texts.append(front_text)


    spanish_text_candidate_1 = remove_html_tags_bs(note.fields[indexes_for_getting_text[0]])
    spanish_text_candidate_2 = remove_html_tags_bs(note.fields[indexes_for_getting_text[1]])

    if spanish_text_candidate_1 == '' or spanish_text_candidate_2 == '':
        print('Проблемная карточка! Пустое поле!')
        print(note.fields)
        raise ValueError

    spanish_text = get_spanish_dominant_string(
        spanish_text_candidate_1,
        spanish_text_candidate_2
    )
    # print(note.keys())

    if 'amarill' in spanish_text:
        print('---')
        print(spanish_text)

    spanish_text = remove_prefix(spanish_text)

    if 'amarill' in spanish_text:
        print(spanish_text)
        print('===')

    anki_texts.append(spanish_text)

# Закрываем коллекцию
col.close()

# Выводим результаты
# for text in anki_front_texts:
#     print(text)

print(f'Выгружено {len(anki_texts)} карточек. Примеры карточек:')

# Определяем количество карточек для вывода
num_to_display = min(len(anki_texts), 20)

# Выбираем случайные карточки
random_cards = random.sample(anki_texts, num_to_display)

# Выводим выбранные карточки
for card in random_cards:
    print(card)

---
amarillo
amarillo
===
---
la luz amarilla
luz amarilla
===
---
pulsar la tecla amarilla
pulsar la tecla amarilla
===
Выгружено 1362 карточек. Примеры карточек:
estar prohibido
batería
parar la hemorragia
¿qué anchura tiene la mesita?
detener
bebida
doce
espejo retrovisor
obligatorio
encontrarse
bicicleta
no sé
contrario
sol
ir al trabajo
hora punta
acostarse
estacionar el vehículo
cuatrocientos ochenta y dos
luces direccionales


/var/folders/0y/334bw9nn6893hng3ljst5tl80000gn/T/ipykernel_40431/3220613337.py:43: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")


In [3]:
load_dotenv()  # Загружает переменные из .env файла в окружение
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

In [4]:

# Загрузим модель испанского языка в spacy
nlp = spacy.load("es_core_news_md")


In [5]:
client = OpenAI(api_key=OPENAI_API_KEY)

# Загрузим модель испанского языка в spacy
nlp = spacy.load("es_core_news_md")

KNOWN_WORDS_FILE = 'known_words.txt'
TRANSLATIONS_FILE = 'translations.txt'
CACHE_FILE = 'translation_cache.json'

POS_NAMES = {
    "ADJ": "прилагательное",
    "ADP": "предлог",
    "ADV": "наречие",
    "AUX": "вспомогательный глагол",
    "CONJ": "союз",
    "CCONJ": "сочинительный союз",
    "DET": "определитель",
    "INTJ": "междометие",
    "NOUN": "существительное",
    "NUM": "числительное",
    "PART": "частица",
    "PRON": "местоимение",
    "PROPN": "собственное имя",
    "PUNCT": "пунктуация",
    "SCONJ": "подчинительный союз",
    "SYM": "символ",
    "VERB": "глагол",
    "X": "неизвестное"
}

# def extract_text_from_html(html):
#     soup = BeautifulSoup(html, 'html.parser')
#     text = ''
#     for element in soup.find_all(['p', 'h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'span', 'div']):
#         if element.string:
#             text += element.string.strip() + ' '
#     return text.strip()

def extract_text_from_html(html):
    # Создаем объект BeautifulSoup
    soup = BeautifulSoup(html, 'html.parser')
    
    # Извлекаем все блоки с классом "col-md-8"
    blocks = soup.find_all('div', class_='col-md-8')
    
    # Извлекаем текст из каждого блока и объединяем их
    text = "\n".join([block.get_text(separator=" ", strip=True) for block in blocks])
    
    return text.strip()

def fetch_page_content(url):
    response = requests.get(url)
    response.raise_for_status()
    return extract_text_from_html(response.text)

def fetch_page_content_local_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return extract_text_from_html(file.read())

def fetch_page_content_local_file_dir(path):
    text = ''
    for filename in os.listdir(path):
        print(f'Processing {filename}')
        if filename.lower().endswith('.html'):
            with open(os.path.join(path, filename), 'r', encoding='utf-8') as file:
                
                extracted_text = extract_text_from_html(file.read())
                if 'colocar él' in extracted_text:
                    print('\tWE FOUND "colocar él"')
                text += extracted_text + '\n'
        else:
            print('\tWE SKIPPED THIS FILE')

    print(f'Text:\n{text}')
    return text





# def normalize_and_count_words(text):
#     doc = nlp(text)
#     lemmatized_words = [f"{token.lemma_.lower()} ({POS_NAMES.get(token.pos_, 'другая часть речи')})" for token in doc if token.is_alpha]
#     return Counter(lemmatized_words)

def normalize_and_count_words(text):
    doc = nlp(text)
    lemmatized_words = []
    for token in doc:
        if token.is_alpha:
            lemma = token.lemma_.lower()
            # Если лемма содержит местоимение в конце, обрезаем его
            if lemma.endswith(" él"):
                lemma = lemma[:-3]  # Удаляем " él"
            # elif lemma.endswith(" ella"):
            #     lemma = lemma[:-5]  # Удаляем " ella"
            # elif lemma.endswith(" nosotros"):
            #     lemma = lemma[:-9]  # Удаляем " nosotros"
            # elif lemma.endswith(" vosotros"):
            #     lemma = lemma[:-9]  # Удаляем " vosotros"
            
            # Добавляем токен, если он не пустой
            if lemma:
                lemmatized_words.append(f"{lemma} ({POS_NAMES.get(token.pos_, 'другая часть речи')})")
    return Counter(lemmatized_words)

def load_known_words():
    if os.path.exists(KNOWN_WORDS_FILE):
        with open(KNOWN_WORDS_FILE, 'r', encoding='utf-8') as file:
            result = set(file.read().splitlines())
            result = result | set(anki_texts)
            return result
    return set()

def save_known_words(known_words):
    with open(KNOWN_WORDS_FILE, 'w', encoding='utf-8') as file:
        file.write('\n'.join(sorted(known_words)))

def load_translations():
    translations = {}
    if os.path.exists(TRANSLATIONS_FILE):
        with open(TRANSLATIONS_FILE, 'r', encoding='utf-8') as file:
            for line in file:
                parts = line.strip().split('\t')
                if len(parts) == 2:
                    word, translation = parts
                    translations[word] = translation
    return translations

def save_translation(word, pos, translation):
    with open(TRANSLATIONS_FILE, 'a', encoding='utf-8') as file:
        pos_str = POS_NAMES.get(pos, 'другая часть речи')
        file.write(f'=[ {word} ({pos_str}) ]='.center(40, '-'))
        file.write(f'\n{translation}\n')
        file.write('-' * 40)
        file.write('\n\n')

def load_cache():
    if os.path.exists(CACHE_FILE):
        with open(CACHE_FILE, 'r', encoding='utf-8') as file:
            return json.load(file)
    return {}

def save_cache(cache):
    with open(CACHE_FILE, 'w', encoding='utf-8') as file:
        json.dump(cache, file)

def get_translation(word, pos, debug=False):
    cache = load_cache()
    cache_key = f"{word}_{pos}"
    print(f"Translating {word} ({pos})")
    
    if cache_key in cache and not debug:
        return cache[cache_key]
    
    with open('question_template.txt', 'r', encoding='utf-8') as file:
        question = file.read()
    question = question.format(word=word, pos=pos)
    if debug:
        print(f"Question for OpenAI: {question}")
        raise ValueError('Debug mode')

    try:
        response = client.chat.completions.create(
            model="gpt-4o",
            # model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": question}
            ]
        )
        translation = response.choices[0].message.content.strip()
        cache[cache_key] = translation
        save_cache(cache)
    except openai.APIError as e:
        #Handle API error here, e.g. retry or log
        print(f"OpenAI API returned an API Error: {e}")
        pass
    except openai.APIConnectionError as e:
        #Handle connection error here
        print(f"Failed to connect to OpenAI API: {e}")
        pass
    except openai.RateLimitError as e:
        #Handle rate limit error (we recommend using exponential backoff)
        print(f"OpenAI API request exceeded rate limit: {e}")
        pass
    return translation

class WordFrequencyChecker(QWidget):
    def __init__(self, source, source_type='url'):
        super().__init__()
        self.known_words = load_known_words()
        # self.known_words = self.known_words | set(anki_front_texts)
        self.translations = load_translations()
        self.initUI(source, source_type)
    
    def initUI(self, source, source_type = 'url'):
        if source_type == 'url':
            text_content = fetch_page_content(source)
        elif source_type == 'file':
            text_content = fetch_page_content_local_file(source)
        elif source_type == 'dir':
            text_content = fetch_page_content_local_file_dir(source)
        else:
            raise ValueError('Unknown source type')
            
        
        word_frequencies = normalize_and_count_words(text_content)
        
        layout = QVBoxLayout()
        layout.addWidget(QLabel('Выберите слова, которые вы знаете'))
        
        self.checkboxes = {}
        
        print(word_frequencies)
        for word, freq in word_frequencies.most_common(1000):  # выводим топ-40 слов
            if freq > 5:
                word_only = word.split(' (')[0]
                if word_only not in self.known_words:
                    checkbox = QCheckBox(f'{word} ({freq})')
                    self.checkboxes[word] = checkbox
                    layout.addWidget(checkbox)
        
        save_button = QPushButton('Сохранить')
        save_button.clicked.connect(self.save_known_words)
        layout.addWidget(save_button)
        
        translate_button = QPushButton('Перевести оставшиеся')
        translate_button.clicked.connect(self.translate_words)
        layout.addWidget(translate_button)
        
        self.translations_output = QLabel('')
        layout.addWidget(self.translations_output)
        
        scroll = QScrollArea()
        container = QWidget()
        container.setLayout(layout)
        scroll.setWidget(container)
        scroll.setWidgetResizable(True)
        
        main_layout = QVBoxLayout()
        main_layout.addWidget(scroll)
        self.setLayout(main_layout)
        
        self.setWindowTitle('Word Frequency Checker')
        self.show()
    
    def save_known_words(self):
        for word, checkbox in self.checkboxes.items():
            if checkbox.isChecked():
                self.known_words.add(word.split(' (')[0])
        save_known_words(self.known_words)
        self.close()
    
    def translate_words(self):
        untranslated_words = [(word.split(' (')[0], token.pos_) for word, token in [(word.split(' (')[0], nlp(word.split(" (")[0])[0]) for word in self.checkboxes if not self.checkboxes[word].isChecked()]]
        translations = {}
        for word, pos in untranslated_words:
            if word in self.translations:
                translations[word] = self.translations[word]
            else:
                translation = get_translation(word, POS_NAMES.get(pos, 'другая часть речи'))
                if translation:
                    print(f'{word}: {translation}')
                    translations[word] = translation
                    save_translation(word, pos, translation)
        
        self.translations_output.setText('\n'.join([f'\n[{word} ({pos})]:\n{translation}' for word, translation in translations.items()]))

class UnknownWordFrequencySaver:
    def __init__(self, source, known_words, source_type='url'):
        self.known_words = set(known_words)
        self.source = source
        self.source_type = source_type
        self.word_frequencies = self.process_text()
    
    def process_text(self):
        if self.source_type == 'url':
            text_content = fetch_page_content(self.source)
        elif self.source_type == 'file':
            text_content = fetch_page_content_local_file(self.source)
        elif self.source_type == 'dir':
            text_content = fetch_page_content_local_file_dir(self.source)
        else:
            raise ValueError('Unknown source type')
        
        words = normalize_and_count_words(text_content)
        return words
    
    def save_to_excel(self, filename):
        total_words = sum(self.word_frequencies.values())
        
        # for word, freq in word_frequencies.most_common(1000):  # выводим топ-40 слов
        #     if freq > 5:
        #         word_only = word.split(' (')[0]
        #         if word_only not in self.known_words:

        data = []
        for word, freq in self.word_frequencies.most_common(1000):
            if freq > 5:
                if word.split(' (')[0] not in self.known_words:
                    frequency = freq / total_words
                    part_of_speech = word.split(' (')[1].strip(')') if ' (' in word else ''
                    data.append([word.split(' (')[0], part_of_speech, frequency, freq])
        
        df = pd.DataFrame(data, columns=['Word', 'Part of Speech', 'Frequency', 'Count'])
        df.to_excel(filename, index=False)
        print(f'Data saved to {filename}')

def main(source, source_type):
    # старая версия с интерфейсом
    # app = QApplication([])
    # ex = WordFrequencyChecker(source, is_url)
    # app.exec_()

    # новая версия
    print('>>>')
    known_words = load_known_words()  # Загрузка известных слов
    for word in known_words:
        if 'amarill' in word:
            print(word)
    print('<<<')
    # source = 'path_to_your_text_file_or_url'
    saver = UnknownWordFrequencySaver(source, known_words, source_type)  # 'url', 'file', 'dir'
    saver.save_to_excel('word_frequencies.xlsx')

In [6]:
normalize_and_count_words('''físico incorporar él.
                          ''')

Counter({'físico (прилагательное)': 1,
         'incorporar (глагол)': 1,
         'él (местоимение)': 1})

In [7]:
# Пример использования
if __name__ == "__main__":
    source = './data/Albina'  # Замените на ваш файл или URL
    source_type = 'dir'  # Установите True, если source это URL
    main(source, source_type)

>>>
pulsar la tecla amarilla
luz amarilla
amarillo
<<<
Processing 2025-02-03 Test del Permiso B Gratis de la DGT con Explicaciones 2025.mhtml
	WE SKIPPED THIS FILE
Processing 07.20_Test del Permiso B Gratis de la DGT con Explicaciones 2024.html
Processing 2025-02-21 Test del Permiso B Gratis de la DGT con Explicaciones 2025.mhtml
	WE SKIPPED THIS FILE
Processing 04-10-2024 Test del Permiso B Gratis de la DGT con Explicaciones 2024.html
Processing 07.23_Test del Permiso B Gratis de la DGT con Explicaciones 2024.html
Processing Test del Permiso B Gratis de la DGT con Explicaciones 2024_24.06.html
Processing 2024-10-29 Test del Permiso B Gratis de la DGT con Explicaciones 2024.html
Processing Test del Permiso B Gratis de la DGT con Explicaciones 2024_12.06.html
Processing 11-10-2024 Test del Permiso B Gratis de la DGT con Explicaciones 2024.html
Processing 2025-01-30 Test del Permiso B Gratis de la DGT con Explicaciones 2025.mhtml
	WE SKIPPED THIS FILE
Processing 2025-03-01 Test del Permi

In [8]:
# # Пример использования
# if __name__ == "__main__":
#     source = './data/Test del Permiso B Gratis de la DGT con Explicaciones 2024.html'  # Замените на ваш файл или URL
#     source_type = 'file'  # Установите True, если source это URL
#     main(source, source_type)